This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [1]:
import sys
sys.path.append('../py')

In [2]:
import tensorflow as tf
import numpy as np
import time
import importlib

In [3]:
import cavitylearn.data
import cavitylearn.catalonet0

In [4]:
importlib.reload(cavitylearn.catalonet0)

<module 'cavitylearn.catalonet0' from '../py/cavitylearn/catalonet0.py'>

In [5]:
config = tf.ConfigProto(
        #device_count = {'GPU': 0}
    )

In [6]:
sess = tf.InteractiveSession(config=config)

In [7]:
dataconfig = cavitylearn.data.DataConfig(classes=["SO4","GOL","EDO","HEM","NAG","PO4","ACT"], num_props=8, boxshape=[30, 30, 30])
with open('/var/cavitylearn/dataset-test/labels.txt', 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, '/var/cavitylearn/dataset-test', dataconfig)

In [8]:
trainset.N

21687

In [9]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])

In [10]:
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [11]:
NUM_FILTERS=12
PIXELS= dataconfig.boxshape[0] * dataconfig.boxshape[1] * dataconfig.boxshape[2]
BATCHSIZE=50

In [12]:
logits = cavitylearn.catalonet0.inference(boxes_placeholder, dataconfig)

In [13]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(labels_placholder * tf.log(logits), reduction_indices=[1]))

In [14]:
sess.run(tf.initialize_all_variables())

In [15]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [16]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))

In [17]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
NUM_BATCHES=100

In [19]:
start_time = time.time()
for i in range(NUM_BATCHES):
    labels, boxes = trainset.next_batch(BATCHSIZE)
    train_step.run(feed_dict={boxes_placeholder: boxes, labels_placholder: labels})
    print(';', str(accuracy.eval(feed_dict={boxes_placeholder: boxes, labels_placholder: labels})), end='')
end_time = time.time()

; 0.22; 0.18; 0.12; 0.18; 0.24; 0.16; 0.24; 0.22; 0.26; 0.2; 0.22; 0.12; 0.18; 0.22; 0.2; 0.2; 0.24; 0.26; 0.24; 0.18; 0.18; 0.26; 0.28; 0.2; 0.04; 0.26; 0.18; 0.2; 0.18; 0.28; 0.22; 0.14; 0.24; 0.2; 0.26; 0.26; 0.22; 0.16; 0.26; 0.34; 0.18; 0.2; 0.38; 0.3; 0.26; 0.22; 0.22; 0.18; 0.14; 0.18; 0.16; 0.18; 0.14; 0.3; 0.24; 0.1; 0.18; 0.26; 0.18; 0.28; 0.22; 0.18; 0.2; 0.16; 0.24; 0.14; 0.16; 0.18; 0.18; 0.2; 0.28; 0.18; 0.14; 0.24; 0.14; 0.28; 0.24; 0.3; 0.06; 0.24; 0.16; 0.24; 0.16; 0.2; 0.26; 0.2; 0.16; 0.14; 0.22; 0.22; 0.12; 0.18; 0.28; 0.2; 0.2; 0.2; 0.14; 0.16; 0.18; 0.22

In [20]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

Total time:  81.55472660064697
Time per batch:  0.8155472660064698
